# JSON Based Data Exercises: World Bank Projects
### Springboard Data Science Career Track (C. Bonfield)

In this notebook, we explore the JSON data contained in   `data/world_bank_projects.json`. 

Our objectives are as follows:
1. Find the ten countries with the most projects.
2. Find the top ten major project themes (using column `mjtheme_namecode`). 
3. Fill in the code names for the entries that are missing them (from \#2). 

## 0. Load/Examine the Data

In [1]:
# Import statements 
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

# Tweaks
pd.set_option('display.max_columns', None)

# Load data
datafile = 'data/world_bank_projects.json'
wbp_data = pd.read_json(datafile)

In [2]:
# Inspect data
wbp_data.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,docty,envassesmentcategorycode,grantamt,ibrdcommamt,id,idacommamt,impagency,lendinginstr,lendinginstrtype,lendprojectcost,majorsector_percent,mjsector_namecode,mjtheme,mjtheme_namecode,mjthemecode,prodline,prodlinetext,productlinetype,project_abstract,project_name,projectdocs,projectfinancialtype,projectstatusdisplay,regionname,sector,sector1,sector2,sector3,sector4,sector_namecode,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,"Project Information Document,Indigenous People...",C,0,0,P129828,130000000,MINISTRY OF EDUCATION,Investment Project Financing,IN,550000000,"[{'Percent': 46, 'Name': 'Education'}, {'Perce...","[{'code': 'EX', 'name': 'Education'}, {'code':...",[Human development],"[{'code': '8', 'name': 'Human development'}, {...","8,11",PE,IBRD/IDA,L,{'cdata': 'The development objective of the Se...,Ethiopia General Education Quality Improvement...,"[{'DocDate': '28-AUG-2013', 'EntityID': '09022...",IDA,Active,Africa,"[{'Name': 'Primary education'}, {'Name': 'Seco...","{'Percent': 46, 'Name': 'Primary education'}","{'Percent': 26, 'Name': 'Secondary education'}","{'Percent': 16, 'Name': 'Public administration...","{'Percent': 12, 'Name': 'Tertiary education'}","[{'code': 'EP', 'name': 'Primary education'}, ...","ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,"Project Information Document,Integrated Safegu...",C,4700000,0,P144674,0,MINISTRY OF FINANCE,Specific Investment Loan,IN,5700000,"[{'Percent': 70, 'Name': 'Public Administratio...","[{'code': 'BX', 'name': 'Public Administration...","[Economic management, Social protection and ri...","[{'code': '1', 'name': 'Economic management'},...","1,6",RE,Recipient Executed Activities,L,NaN,TN: DTF Social Protection Reforms Support,"[{'DocDate': '29-MAR-2013', 'EntityID': '00033...",OTHER,Active,Middle East and North Africa,[{'Name': 'Public administration- Other social...,"{'Percent': 70, 'Name': 'Public administration...","{'Percent': 30, 'Name': 'General public admini...",NaN,NaN,"[{'code': 'BS', 'name': 'Public administration...","BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,"Resettlement Plan,Environmental Assessment,Int...",B,0,0,P145310,6060000,MINISTRY OF TRANSPORT AND COMMUNICATIONS,Investment Project Financing,IN,6060000,"[{'Percent': 100, 'Name': 'Transportation'}]","[{'code': 'TX', 'name': 'Transportation'}]","[Trade and integration, Public sector governan...","[{'code': '5', 'name': 'Trade and integration'...","5,2,11,6",PE,IBRD/IDA,L,NaN,Tuvalu Aviation Investment Project - Additiona...,"[{'DocDate': '21-OCT-2013', 'EntityID': '00033...",IDA,Active,East Asia and Pacific,[{'Name': 'Rural and Inter-Urban Roads and Hig...,"{'Percent': 100, 'Name': 'Rural and Inter-Urba...",NaN,NaN,NaN,"[{'code': 'TI', 'name': 'Rural and Inter-Urban...",TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17

Nifty! Let's dive right into the data. From the rows displayed above, it looks as though we have a single project (with all associated information) per row. The relevant columns for our work will be:

* `countryname`: name of country listed on project
* `mjtheme_namecode`: major project theme(s) for a given project

## 1. Find the ten countries with the most projects. 

To find the ten countries with the most projects, we need to just tally up the number of rows in which each country appears and select the top ten. We can do this without too much trouble using `Series.value_counts()` on the `countryname` column, explicity providing `ascending=False` to return our list in descending order (though this is the default option). Once we have the full list, we can display the top ten using `.head(10)`. 

In [3]:
num_projects_by_country = wbp_data.countryname.value_counts().sort_values(ascending=False)
num_projects_by_country.head(10)

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

## 2. Find the top ten major project themes (using column `mjtheme_namecode`). 

Prior to attempting this step, let's examine the value contained in the `mjtheme_namecode` column in the first row. 

In [4]:
print(wbp_data.mjtheme_namecode[0])
type(wbp_data.mjtheme_namecode[0])

[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]


list

The contents of `mjtheme_namecode` are nested JSON strings! Therefore, we must do a little extra to tease out the major project themes from that column. Additionally, it is worth noting that more than one project theme may appear for a given project.

The work to be done here is as follows:
* Treat nested JSON strings using `json_normalize`.
* Count number of times `code`/`name` appear in normalized dataframe (I will do this both ways, motivating third problem). 
* Display results.

In [5]:
# Reload JSON data as string (allows for direct treatment of nested element.)
with open(datafile) as json_file:
    json_data = json.load(json_file)

themes_norm = json_normalize(json_data, 'mjtheme_namecode')
themes_norm.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


We note that there are some missing elements in the `name` column - we will treat these in the next stage of the problem. For now, let's find the ten most popular project themes (by `name` and by `code`) and list all `name`/`code` pairs to check for consistency. 

In [6]:
print('BY CODE:')
code_pop_themes = themes_norm.code.value_counts().sort_values(ascending=False)
print(code_pop_themes.head(10))

BY CODE:
11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64


In [7]:
print('BY NAME:')
name_pop_themes = themes_norm.name.value_counts().sort_values(ascending=False)
print(name_pop_themes.head(10))

BY NAME:
Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64


In [8]:
# Ignore indices (treats code as string, resulting in 10/11 coming after 1 and before 2). 
print('LIST OF CODE/NAME PAIRS:')
print(themes_norm[themes_norm.name != ''].sort_values(by='code').drop_duplicates().reset_index().drop('index', axis=1))

LIST OF CODE/NAME PAIRS:
   code                                          name
0     1                           Economic management
1    10                             Rural development
2    11  Environment and natural resources management
3     2                      Public sector governance
4     3                                   Rule of law
5     4      Financial and private sector development
6     5                         Trade and integration
7     6         Social protection and risk management
8     7                   Social dev/gender/inclusion
9     8                             Human development
10    9                             Urban development


Although the rankings are consistent, we note that the counts are indeed different, indicating that we could have run into trouble if we had a lot of missing `name` data for any given `code` (sufficient motivation for only counting with columns that are complete!). 

## 3. Fill in the code names for the entries that are missing them (from \#2). 

Now, let's fill in the missing names. To do so, we can just sort the normalized dataframe on `code`, then forward fill. 

In [9]:
# Check how many values are missing initially. 
print('Number of missing names: %d' % themes_norm[themes_norm.name == '']['name'].count())

Number of missing names: 122


In [10]:
# Replace missing values with NaNs, then fill with appropriate name using ffill/bfill.
#
# NOTE: For the first row with code=11, name is blank. To catch that value, we have to backwards fill 
#       after forward filling (I only noticed this was true after checking the results.)
themes_norm['name'] = themes_norm.name.apply(lambda x : np.NaN if len(x) == 0 else x)
themes_norm['name'] = themes_norm.groupby('code')['name'].fillna(method='ffill')
themes_norm['name'] = themes_norm.groupby('code')['name'].fillna(method='bfill')

In [11]:
# Check if counts on name are now consistent with those we found when counting on code above.
rev_name_pop_themes = themes_norm.name.value_counts().sort_values(ascending=False)
print(rev_name_pop_themes.head(10))

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64


In [12]:
# Check how many values are missing after filling (better be zero!). 
print('Number of missing names: %d' % themes_norm[themes_norm.name == '']['name'].count())

Number of missing names: 0


That finishes our exploration with JSON data! Happy coding!